<a href="https://colab.research.google.com/github/Ilyaas189/CE-888-Assignment-2/blob/main/Hate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
import os
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
lemma = WordNetLemmatizer()
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


**Loading the train and test data**

In [2]:
with open("train_text.txt" ,encoding="utf8") as f:
    X_train = f.readlines()
with open('train_labels.txt',encoding="utf8") as f:
    y_train = f.readlines()
with open('test_text.txt',encoding="utf8") as f:
    X_test = f.readlines()
with open('test_labels.txt',encoding="utf8") as f:
    y_test = f.readlines()

**To determine the size of the X and Y parameters of Train and test data**

In [3]:
print("X test:",len(X_test),"X train:",len(X_train))
print("y test",len(y_test),"y train",len(y_train))

X test: 2970 X train: 9000
y test 2970 y train 9000


**Cleaning the datasets**

In [4]:
def clean_text(text):
    totalStopwords = set([word.replace("'",'') for word in stopwords.words('english')])
    text = text.lower()
    text = text.replace("'",'')
    text = re.sub('[^a-zA-Z]',' ',text)
    words = text.split()
    words = [lemma.lemmatize(word) for word in words if (word not in totalStopwords) and (len(word)>1)] # Remove stop words
    text = " ".join(words)
    return text

**Mapping the cleaned datasets**

In [5]:
X_train = map(clean_text, X_train)
X_test = map(clean_text, X_test)

**For the Model input - Configuring  the datasets into a list**

In [6]:
X_train = list(X_train)
X_test = list(X_test)

**Encoding each word into a label**

In [7]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

**Vectorizing the Datasets**

In [8]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

**Training the SVC on training dataset**

In [9]:
from sklearn.svm import SVC

model = SVC()
model.fit(X_train, y_train)

# print prediction results
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.73      0.18      0.29      1718
           1       0.45      0.91      0.60      1252

    accuracy                           0.49      2970
   macro avg       0.59      0.54      0.44      2970
weighted avg       0.61      0.49      0.42      2970



**Accuracy Score on default values**

In [10]:
accuracy_score(y_test, predictions)

0.48754208754208755

**Employing the Grid Search Corss Validation to select the best parameters for optimum accuracy**

In [11]:
from sklearn.model_selection import GridSearchCV
  
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.609, total=  10.0s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.0s remaining:    0.0s


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.611, total=   9.9s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   19.9s remaining:    0.0s


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.611, total=   9.8s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.612, total=   9.9s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.616, total=   9.8s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.582, total=   9.2s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.583, total=   9.5s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.582, total=   9.3s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.581, total=   9.5s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] .

[Parallel(n_jobs=1)]: Done 125 out of 125 | elapsed: 20.2min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

**Printing the best parameters and estimates**

In [12]:
# print best parameter after tuning
print(grid.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

{'C': 10, 'gamma': 1, 'kernel': 'rbf'}
SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf', max_iter=-1,
    probability=False, random_state=None, shrinking=True, tol=0.001,
    verbose=False)


**Prediction of the best scores using the Grid Search Algorithm model**

In [13]:
grid_predictions = grid.predict(X_test)
  
# print classification report
print(classification_report(y_test, grid_predictions))

              precision    recall  f1-score   support

           0       0.76      0.19      0.31      1718
           1       0.45      0.92      0.61      1252

    accuracy                           0.50      2970
   macro avg       0.61      0.55      0.46      2970
weighted avg       0.63      0.50      0.43      2970



**Optimum Acccuracy after employing the Grid Search parameters**

In [14]:
accuracy_score(y_test, grid_predictions)

0.49764309764309766